In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import tensorflow as tf
from scipy.misc import imread
import matplotlib
%matplotlib inline

metadata = pd.read_csv('../input/metadata.csv')

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
# Any results you write to the current directory are saved as output.

metadata.csv
train
train_masks



In [2]:
import pickle

def _batch_image(file):
    image = imread(file)/255
    mask_file = file.split('/')[-1].split('.')[0] + "_mask.gif"
    mask_file = os.path.join('../input/train_masks', mask_file)
    mask = imread(mask_file)/255
    return image, mask

train_files = np.array([os.path.join("../input/train", f) for f in os.listdir("../input/train")])

if not os.path.exists("../batches"):
    os.makedirs("../batches")
    batch_file_size = int(len(train_files)/50)

    count = 1
    for batch in range(batch_file_size, len(train_files), batch_file_size):
        batch_file = os.path.join("../batches", str(count)+".pickle")
        batch_images = np.zeros((batch_file_size, 1280, 1918, 3), dtype=np.float32)
        batch_masks = np.zeros((batch_file_size, 1280, 1918, 3), dtype=np.float32)

        start = batch - batch_file_size
        end = batch
        for i, file in enumerate(train_files[start:end]):
            batch_images[i], batch_masks[i] = _batch_image(file)
        
        with open(batch_file, 'w+') as file:
            pickle.dump({"images": batch_images, "masks": batch_masks}, file)

        count += 1

TypeError: write() argument must be str, not bytes

In [ ]:
# Hyperparams
batch_size = 50
iterations = 1000

# Start a new session to show example output.
sess = tf.Session()
# Required to get the filename matching to run.
sess.run(tf.global_variables_initializer())
for i in range(iterations):
    indxs = np.random.choice(len(train_files), batch_size)
    batch_files = train_files[indxs]
    batch_images = np.zeros((len(batch_files), 1280, 1918, 3), dtype=np.float32)
    batch_labels = np.zeros((len(batch_files), 1280, 1918, 3), dtype=np.float32)
    for i, file in enumerate(batch_files):
        batch_images[i] = imread(file)/255
        mask_file = file.split('/')[-1].split('.')[0] + "_mask.gif"
        mask_file = os.path.join('../input/train_masks', mask_file)
        batch_labels[i] = imread(mask_file)/255
